In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [4]:
EXPERIMENT_FAMILY_NAME = 'torch'
EXPERIMENT_NAME = 'deeppavlov'
N_RANDOM_SEEDS = 1
N_EPOCHS = 100
BATCH_SIZE = 64

In [5]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [6]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [7]:

train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl'))
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl'))

train = train.rename(columns={cfg.TEXT_COL: 'text'})
test = test.rename(columns={cfg.TEXT_COL: 'text'})

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

# train = filter_rare_categories(train)

In [8]:
# EMB_NAME = 'smaller_LaBSE_15lang'
# train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'train.pkl'))
# test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'test.pkl'))

# train = train.join(train_emb)
# test = test.join(test_emb)

# del train_emb, test_emb; gc.collect()

In [9]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]
y_train_onecol = y_train.copy()

# y_train = pd.get_dummies(y_train)

N_OUT_CLASSES = len(CLASSES)
# assert y_train.shape[1] == N_OUT_CLASSES

In [10]:
theme_train = pd.read_csv(cfg.ORIG_TRAIN_PATH, index_col=cfg.ID_COL, usecols=[cfg.ID_COL, cfg.THEME_COL])
theme_train = theme_train[cfg.THEME_COL].astype('category').cat.codes
N_THEMES = theme_train.nunique()
# theme_train = pd.get_dummies(theme_train)
# theme_train.columns = [f'theme_{i}' for i in range(theme_train.shape[1])]


In [11]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [12]:
from typing import Optional


def df_to_dataset(data: pd.DataFrame, labels: Optional[pd.DataFrame]=None, shuffle: bool=True, batch_size: int=32, labels2: Optional[pd.DataFrame]=None):
  df = data.copy()
  df = {key: value.values[:, tf.newaxis] for key, value in data.items()}
  
  if labels is None:
    ds = tf.data.Dataset.from_tensor_slices((dict(df),))
  else:
    ds = tf.data.Dataset.from_tensor_slices((
      dict(df), 
      {
        'category_out': labels, 
        'theme_out': labels2
        }
        ))
        
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds


def get_keras_input(
    data: pd.DataFrame,
    labels: Optional[pd.DataFrame] = None,
    labels2: Optional[pd.DataFrame] = None,
    shuffle=False
    ) -> tf.data.Dataset:
    return df_to_dataset(
        data=data,
        labels=labels,
        labels2=labels2,
        shuffle=shuffle, 
        batch_size=BATCH_SIZE)
        


In [13]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1", name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1", trainable=False, name='encoder')
    outputs = encoder(encoder_inputs)

    pooled_out = outputs['pooled_output']
    seq_out = outputs['sequence_output']
    rnn_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, dropout=0.1, activation='relu'))(seq_out)

    pooled_max = tf.keras.layers.GlobalMaxPool1D()(seq_out)
    net = tf.keras.layers.Concatenate()([pooled_out, pooled_max, rnn_out])
    
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(512)(net)
    net = tf.keras.layers.LeakyReLU()(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(256)(net)
    net = tf.keras.layers.LeakyReLU()(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    category_out = tf.keras.layers.Dense(N_OUT_CLASSES, activation=None, name='category_out')(net)
    theme_out = tf.keras.layers.Dense(N_THEMES, activation=None, name='theme_out')(net)

    outputs = {
        'category_out': category_out, 
        'theme_out': theme_out
        }
    model = tf.keras.Model(
        text_input, 
        outputs=outputs
        )

    loss = {
        'category_out': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='category_out'),
        'theme_out': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='theme_out')
        }
    # metrics = tf.keras.metrics.AUC(from_logits=True, name='auc')

    epochs = N_EPOCHS
    n = (len(train) // cfg.N_SPLITS) * (cfg.N_SPLITS-1)
    steps_per_epoch = n // BATCH_SIZE
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-4
    optimizer = optimization.create_optimizer(
        init_lr=init_lr,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw')

    model.compile(
        optimizer=optimizer,
        loss=loss,
        loss_weights={"category_out": 2.0, "theme_out": 0.35}
        #  metrics=metrics
        )
    return model
    

In [14]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
leak_mask = leak_test.notnull()

In [15]:
cv = StratifiedKFold(n_splits=cfg.N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = get_keras_input(
        data=test,
        shuffle=False
        )


metrics = defaultdict(list)
fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, y_train_onecol), total=cfg.N_SPLITS):

    train_pool = get_keras_input(
        data=X_train.iloc[train_idx], 
        labels=y_train.iloc[train_idx],
        labels2=theme_train.iloc[train_idx],
        shuffle=True
        )

    val_pool = get_keras_input(
        data=X_train.iloc[val_idx], 
        labels=y_train.iloc[val_idx],
        labels2=theme_train.iloc[val_idx],
        shuffle=False
        )
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = build_classifier_model()
        
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_category_out_loss', patience=3, min_delta=1e-5, mode='auto', restore_best_weights=True)
        clf.fit(x=train_pool, validation_data=val_pool, epochs=N_EPOCHS, callbacks=[early_stopping])
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save(os.path.join(model_path, model_name), include_optimizer=False)
        
        pred_proba_oof_val = tf.nn.softmax(clf.predict(val_pool)['category_out']).numpy()
        pred_proba_oof_train = tf.nn.softmax(clf.predict(train_pool)['category_out']).numpy()
        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += tf.nn.softmax(clf.predict(test_pool)['category_out']).numpy()

        y_val_oof = y_train_onecol.iloc[val_idx]

        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['val_auc'].append(val_auc)
        print('val auc', val_auc)

        leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask], pred_proba_test.loc[leak_mask] / (fold + 1), multi_class='ovo', labels=CLASSES)
        metrics['leak_test_auc_score'].append(leak_test_auc_score)
        print('leak_test_auc_score', leak_test_auc_score)
        
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (cfg.N_SPLITS * N_RANDOM_SEEDS)

2022-07-23 00:47:02.826597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 00:47:02.963624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 00:47:02.964024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 00:47:02.966772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
25/25 [==============================] - 101s 3s/step - loss: 15.0025 - category_out_loss: 6.2188 - theme_out_loss: 7.3283 - val_loss: 9.2909 - val_category_out_loss: 3.6580 - val_theme_out_loss: 5.6424
Epoch 2/100
25/25 [==============================] - 70s 3s/step - loss: 13.7260 - category_out_loss: 5.6123 - theme_out_loss: 7.1465 - val_loss: 7.6290 - val_category_out_loss: 2.8371 - val_theme_out_loss: 5.5850
Epoch 3/100
25/25 [==============================] - 70s 3s/step - loss: 11.5098 - category_out_loss: 4.5240 - theme_out_loss: 7.0337 - val_loss: 6.0148 - val_category_out_loss: 2.0417 - val_theme_out_loss: 5.5182
Epoch 4/100
25/25 [==============================] - 72s 3s/step - loss: 9.4754 - category_out_loss: 3.5294 - theme_out_loss: 6.9043 - val_loss: 5.8489 - val_category_out_loss: 1.9724 - val_theme_out_loss: 5.4404
Epoch 5/100
25/25 [==============================] - 64s 3s/step - loss: 8.1920 - category_out_loss: 2.9284 - theme_out_loss: 6.6723 - val_loss:

...

Epoch 1/100
25/25 [==============================] - 86s 3s/step - loss: 11.2355 - category_out_loss: 4.4251 - theme_out_loss: 6.8148 - val_loss: 7.6212 - val_category_out_loss: 2.8430 - val_theme_out_loss: 5.5290
Epoch 2/100
25/25 [==============================] - 64s 3s/step - loss: 10.6619 - category_out_loss: 4.1618 - theme_out_loss: 6.6811 - val_loss: 6.6955 - val_category_out_loss: 2.3871 - val_theme_out_loss: 5.4894
Epoch 3/100
25/25 [==============================] - 66s 3s/step - loss: 9.4295 - category_out_loss: 3.5356 - theme_out_loss: 6.7383 - val_loss: 5.9769 - val_category_out_loss: 2.0372 - val_theme_out_loss: 5.4358
Epoch 4/100
25/25 [==============================] - 65s 3s/step - loss: 8.2847 - category_out_loss: 2.9852 - theme_out_loss: 6.6123 - val_loss: 5.5773 - val_category_out_loss: 1.8487 - val_theme_out_loss: 5.3710
Epoch 5/100
25/25 [==============================] - 63s 3s/step - loss: 7.6242 - category_out_loss: 2.6725 - theme_out_loss: 6.5118 - val_loss: 5

16/16 [==============================] - 25s 1s/step
val auc 0.8167228941238655
leak_test_auc_score 0.8153400312179455


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
25/25 [==============================] - 89s 3s/step - loss: 11.6897 - category_out_loss: 4.6643 - theme_out_loss: 6.7459 - val_loss: 7.9884 - val_category_out_loss: 3.0410 - val_theme_out_loss: 5.4466
Epoch 2/100
25/25 [==============================] - 67s 3s/step - loss: 10.7048 - category_out_loss: 4.1616 - theme_out_loss: 6.8043 - val_loss: 6.9199 - val_category_out_loss: 2.5109 - val_theme_out_loss: 5.4232
Epoch 3/100
25/25 [==============================] - 67s 3s/step - loss: 9.3764 - category_out_loss: 3.5100 - theme_out_loss: 6.7327 - val_loss: 6.1245 - val_category_out_loss: 2.1184 - val_theme_out_loss: 5.3938
Epoch 4/100
25/25 [==============================] - 66s 3s/step - loss: 8.2272 - category_out_loss: 2.9463 - theme_out_loss: 6.6702 - val_loss: 5.5238 - val_category_out_loss: 1.8261 - val_theme_out_loss: 5.3473
Epoch 5/100
25/25 [==============================] - 65s 3s/step - loss: 7.4977 - category_out_loss: 2.6120 - theme_out_loss: 6.4963 - val_loss: 5

16/16 [==============================] - 22s 1s/step
val auc 0.7753700224333154
leak_test_auc_score 0.8172636787596154


In [16]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)
print('scores', metrics['val_auc'])
print('mean', np.mean(metrics['val_auc']), 'std', np.std(metrics['val_auc']))

oof_auc_score 0.7723067434631146
scores [0.7810713984342366, 0.8120336770087735, 0.8665268796973151, 0.8167228941238655, 0.7753700224333154]
mean 0.8103449743395013 std 0.032496540044246715


In [17]:
# 0.8117437772540339

In [18]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
notnull = leak_test.notnull()
leak_test_auc_score = roc_auc_score(leak_test.loc[notnull], pred_proba_test.loc[notnull], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


leak_test_auc_score 0.8172636787596154


In [19]:
# 0.8627341970044778

In [20]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))